In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
from src.driver.service import DriverService
from src.state_manager.state import StateManager
from src.actions.browser_actions import BrowserActions
from src.agent_interface.planing_agent import PlaningAgent
from src.actions.browser_actions import ActionResult
from src.state_manager.utils import save_formatted_html

import os
from datetime import datetime

In [194]:
driver = DriverService().get_driver()
state_manager = StateManager(driver)
actions = BrowserActions(driver)

In [195]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
run_folder = f'temp/run_{timestamp}'
if not os.path.exists(run_folder):
    os.makedirs(run_folder)

In [196]:
task = 'Go to kayak.com and find a flight from Zürich to (ask the user for the destination) on 2025-04-25 with return on 2025-06-05 for 2 people.'

In [197]:
print(actions.get_default_actions())

{'ask_user': 'text: string', 'send_user_text': 'text: string', 'search_google': 'text: string', 'go_to_url': 'url: string', 'done': '', 'go_back': '', 'click': 'id: int', 'input': 'id: int, text: string', 'nothing': ''}


In [198]:
agent = PlaningAgent(default_actions=actions.get_default_actions(), model="gpt-4o")

In [199]:
agent.add_user_prompt(f'Your task is: {task}', after_system=True)

In [200]:
url_history = []
output = ActionResult()

In [201]:
i = 0

In [202]:
previous_state = None

---

In [263]:
current_state = state_manager.get_current_state()

In [264]:
save_formatted_html(driver.page_source, f'{run_folder}/html_{i}.html')

In [265]:
url_history.append(driver.current_url)
text = f'Elements: {state_manager.get_compared_elements(current_state, previous_state)}, Url history: {url_history}'

In [266]:
print(text)

Elements: 2:<div aria-label="Open main navigation"></div>
3:<a href="/" aria-label="Go to the kayak homepage"></a>
4:<div aria-label="Open Trips drawer"></div>
5:<div>Sign in</div>
6:<span>Sign in</span>
372:<div aria-haspopup="listbox" aria-expanded="false" aria-label="Trip type Round-trip">Round-trip</div>
373:<span>Round-trip</span>
375:<div>0 bags</div>
376:<span>0 bags</span>
378:<div>ISB</div>
380:<div aria-label="Remove value"></div>
381:<div>ZRH</div>
383:<div aria-label="Remove value"></div>
384:<input type="text" aria-readonly="false" placeholder="" aria-label="Flight origin input" aria-autocomplete="list" aria-haspopup="listbox" value="" aria-controls="flight-origin-smarty-input-list"></input>
385:<div aria-label="Swap departure airport and destination airport"></div>
(+) 386:<div>Denpasar, Bali, Indonesia (DPS)</div>
388:<div aria-label="Remove value"></div>
390:<div aria-label="Departure">Departure</div>
391:<span>Departure</span>
393:<div aria-label="Return">Return</div>


In [ ]:
if output.user_input:
    agent.add_user_prompt(output.user_input)
if output.error:
    text += f', Previous action error: {output.error}'
action = agent.chat(text, store_conversation=f'{run_folder}/conversation_{i}.txt')

In [ ]:
action

Action(valuation_previous_goal="Successfully input 'Bali' in the destination field and suggestions appeared.", goal="Select 'Denpasar, Bali, Indonesia' from destination suggestions.", action='click', params=ActionParams(url=None, id=2706, text=None))

In [ ]:
output = actions.execute_action(action, current_state.selector_map)

{
    "valuation_previous_goal": "Successfully input 'Bali' in the destination field and suggestions appeared.",
    "goal": "Select 'Denpasar, Bali, Indonesia' from destination suggestions.",
    "action": "click",
    "params": {
        "url": null,
        "id": 2706,
        "text": null
    }
}


In [ ]:
if output.done:
    print('Task completed')
    driver.quit()

In [ ]:
i += 1

In [ ]:
previous_state = current_state

---

In [268]:
driver.quit()